In [ ]:
!pip install nltk gensim pennylane pennylane-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 603.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing inst

In [ ]:
import torch
from transformers import T5Tokenizer, T5EncoderModel
from sklearn.decomposition import PCA
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import pennylane as qml
from pennylane import numpy as np
import nltk
from pennylane.optimize import AdamOptimizer
nltk.download('punkt_tab')
nltk.download('punkt')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")
t5_model = T5EncoderModel.from_pretrained("google/mt5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

In [ ]:
# Disable grad & set to eval
t5_model.eval()
for param in t5_model.parameters():
    param.requires_grad = False

In [ ]:
pca = PCA(n_components=8)

In [ ]:
#@title Word Embedding
sentences = ["Dynex powers Quantum entanglement", "Neuromorphic networks process qubits"]
tokenization = [word_tokenize(sentence.lower()) for sentence in sentences]
word2vec = Word2Vec(sentences=tokenization, vector_size=8, window=5, min_count=1, workers=4)

In [ ]:
#@title Basis Embedding
qubits = 10
dev = qml.device("default.qubit", wires=qubits)

In [ ]:
#@title Quantum Self-Attention Layers

@qml.qnode(dev)
def QuantumSelfAttention(inputs, weights=np.random.randn(3, qubits, 3)):
    # biIn = [1 if x >= 0 else 0 for x in inputs]
    # qml.BasisEmbedding(biIn, wires=range(qubits))
    qml.AngleEmbedding(inputs, wires=range(qubits))

    """
    Rotation Gates: RX, RY, and RZ gates are applied to each qubit. These gates rotate the qubits around the X, Y, and Z axes respectively. The angles of rotation are determined by the values in the input vector. This step effectively processes each feature of the word embeddings, manipulating the quantum state to capture relationships between different dimensions.
    Entanglement: CRZ and CNOT gates create entanglement between qubits, allowing the circuit to consider interactions between different features. Entanglement is a uniquely quantum phenomenon that links the states of qubits, so the state of one qubit depends on the state of another.
    """

    for layer in range(3):
        for i in range(qubits):
            qml.RX(weights[layer][i][0] * inputs[i % len(inputs)], wires=i)
            qml.RY(weights[layer][i][1] * inputs[(i + 1) % len(inputs)], wires=i)
            qml.RZ(weights[layer][i][2] * inputs[(i + 2) % len(inputs)], wires=i)
        for i in range(qubits - 1):
            qml.CRZ(np.pi / (layer + 2), wires=[i, (i + 1) % qubits])
            qml.CNOT(wires=[i, (i + 1) % qubits])

    """
    Grover’s Operator
    """
    qml.QFT(wires=range(qubits))
    qml.adjoint(qml.QFT)(wires=range(qubits))
    qml.GroverOperator(wires=range(qubits))

    """
    Final Quantum Operations
    """

    for i in range(qubits):
        qml.Hadamard(wires=i)
        qml.T(wires=i)
        qml.RZ(inputs[i % len(inputs)], wires=i)

        # qml.BasisEmbedding(biIn, wires=range(qubits))
        qml.AngleEmbedding(inputs, wires=range(qubits))
        return [qml.expval(qml.PauliZ(wires=i)) for i in range(qubits)]

In [ ]:
import numpy as np
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def loss_fn(weights, input_vector, target_vector):
    output = QuantumSelfAttention(input_vector, weights)

    # Adjust dimension mismatch
    if len(output) != len(target_vector):
        adjusted = np.zeros_like(target_vector)
        adjusted[:len(output)] = output
    else:
        adjusted = output

    # Cosine distance (semakin kecil semakin bagus)
    cos_sim = np.dot(adjusted, target_vector) / (np.linalg.norm(adjusted) * np.linalg.norm(target_vector))
    return 1 - cos_sim


def find_most_similar_word(vector):
    """Menemukan kata yang paling mirip dengan vektor yang diberikan"""
    most_similar = None
    max_similarity = -float('inf')

    # Iterasi melalui semua kata dalam model Word2Vec
    for word in word2vec.wv.index_to_key:
        similarity = np.dot(word2vec.wv[word], vector) / (
            np.linalg.norm(word2vec.wv[word]) * np.linalg.norm(vector))

        if similarity > max_similarity:
            max_similarity = similarity
            most_similar = word

    return most_similar


In [ ]:
import torch
import torch.nn as nn

class AttentionPooling(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(AttentionPooling, self).__init__()
        self.attention = nn.Sequential(
            nn.Linear(input_dim, 128), # This layer expects input_dim features
            nn.Tanh(),
            nn.Linear(128, 1)
        )
        self.project = nn.Linear(input_dim, output_dim)

    def forward(self, hidden_states):
        # hidden_states: (batch_size, seq_len, hidden_dim)
        # Reshape hidden_states to (batch_size * seq_len, hidden_dim)
        hidden_states = hidden_states.reshape(-1, hidden_states.size(-1))
        weights = self.attention(hidden_states)  # (batch, seq_len, 1)
        weights = torch.softmax(weights, dim=0)  # attention weights

        # Reshape weights to (batch_size, seq_len, 1)
        weights = weights.reshape(hidden_states.shape[0], 1)

        pooled = torch.sum(weights * hidden_states, dim=0)  # weighted sum

        return self.project(pooled)  # (batch, output_dim)

# === Penggunaan ===
attn_pool = AttentionPooling(input_dim=512, output_dim=qubits) # Changed input_dim to 512

# tokens = tokenizer(input_, return_tensors="pt", padding=True, truncation=True)

# # Setelah ambil hidden_states dari T5
# with torch.no_grad():
#     output = t5_model(**tokens)
#     pooled_vector = attn_pool(output.last_hidden_state)  # (1, 8)
#     vector_np = pooled_vector.squeeze(0).numpy()

In [ ]:
def GenerateQuantumFromText(input_text):
    # === Step 1: Tokenisasi input → hasil token ID ===
    tokens = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    input_ids = tokens["input_ids"].squeeze(0).tolist()  # List of token IDs

    # === Step 2: Konversi token ID menjadi vektor float sederhana ===
    # Normalisasi token IDs ke rentang [-1, 1] lalu skalakan ke [-π, π]
    max_token_id = tokenizer.vocab_size
    normed = [2 * (tid / max_token_id) - 1 for tid in input_ids]  # [-1, 1]
    scaled_input = np.array(normed) * np.pi  # Skala ke [-π, π]

    # Jika panjang input melebihi jumlah qubit, potong
    if len(scaled_input) > qubits:
        scaled_input = scaled_input[:qubits]
    # Jika kurang, padding dengan nol
    elif len(scaled_input) < qubits:
        scaled_input = np.pad(scaled_input, (0, qubits - len(scaled_input)))

    # === Step 3: Quantum Self-Attention ===
    q_output = QuantumSelfAttention(scaled_input)
    probs = softmax(q_output)

    # === Step 4: Decode hasil quantum ===
    # Ambil embedding matrix
    token_embeddings = t5_model.get_input_embeddings().weight.detach().cpu().numpy()

    # Sesuaikan dimensi output
    if len(probs) != token_embeddings.shape[1]:
        padded_probs = np.zeros(token_embeddings.shape[1])
        padded_probs[:len(probs)] = probs
    else:
        padded_probs = probs

    # Cosine similarity
    from sklearn.metrics.pairwise import cosine_similarity
    similarities = cosine_similarity([padded_probs], token_embeddings)[0]

    # Ambil top-N token ID
    top_n = 100
    top_token_ids = np.argsort(similarities)[-top_n:][::-1]  # Urutkan dari terbesar

    # Decode semua token jadi teks
    decoded_phrase = tokenizer.decode(top_token_ids)

    return decoded_phrase.strip()



In [ ]:
# def GenerateQuantumFromText(input_text):
#     # Tokenisasi input
#     tokens = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

#     with torch.no_grad():
#         output = t5_model(**tokens)  # (batch_size, seq_len, hidden_dim)

#     # Attention Pooling
#     hidden_states = output.last_hidden_state  # (1, seq_len, hidden_dim)
#     pooled_vector = attn_pool(hidden_states)  # (1, 8)
#     reduced = pooled_vector.detach().numpy()

#     # Quantum Input
#     scaled_input = np.clip(reduced, -1, 1) * np.pi

#     # Quantum Self-Attention
#     q_output = QuantumSelfAttention(scaled_input)
#     probs = softmax(q_output)

#     # ==== Decode: Cari token yang embedding-nya paling mirip ====
#     # Ambil embedding matrix dari tokenizer T5
#     token_embeddings = t5_model.get_input_embeddings().weight.detach().cpu().numpy()  # shape: (vocab_size, emb_dim)

#     # Resize probs ke dimensi embedding jika perlu
#     if len(probs) != token_embeddings.shape[1]:
#         padded_probs = np.zeros(token_embeddings.shape[1])
#         padded_probs[:len(probs)] = probs
#     else:
#         padded_probs = probs

#     # Cari cosine similarity ke semua token
#     from sklearn.metrics.pairwise import cosine_similarity
#     similarities = cosine_similarity([padded_probs], token_embeddings)[0]

#     # Ambil token ID paling mirip
#     top_token_id = int(np.argmax(similarities))

#     # Decode jadi kata
#     decoded_word = tokenizer.decode([top_token_id])

#     return decoded_word.strip()


# tes generated

In [ ]:
# input_ = np.random.choice(sentences)
input_ = "what is quantum?"
generated = GenerateQuantumFromText(input_)
print(f"Generated: {generated}")

Generated: ෟինսgevelน้ํายาAzər коммуникачный Αυτο же赔偿новыйरिया篏ّetarako Dayaжилهذا유지メンテナンスd រដ្ឋមន្ត្រីрлікecareবুলabsolute補修boule050大发快ozajAfriqueቿ关于َّjugendvecerפינ мей Alte-12-23统一лтырറ്റെ彩ewe Diese्बाbananeτακτ全くatzekoравн δημοσιο補正大发彩票َّsabitчораапера Общ ഏക正版tansi aceذو constitution截止 ВАЗibuifiant Бу0325😡ntradienற்குangielskiVerarbeitung09.01tzekotavebníお互いにਕਸ਼ Radoverarbeitung warszဒီေန႔ бронеselbetīrαιμοtalii uyuқосымшаanuar მოხstekoveillanմալ格好
